In [1]:
import os
os.chdir('/content/drive/MyDrive/yeardream_competition/main2_NLP_myself')
!pwd

/content/drive/MyDrive/yeardream_competition/main2_NLP_myself


In [2]:
import pandas as pd

import re
import ast

# import torch
# from torch.utils.data import Dataset, DataLoader
# from transformers import AutoModelForCausalLM, AutoTokenizer

import pickle
import json

# Train Data

## Train Data 기본 전처리

In [3]:

train = pd.read_csv('train_data.csv')

# cluster 컬럼, 선택지를 리스트로 제대로 담을 컬럼 추가
train['cluster'] = 0
train['__선택지'] = '0'

# 선택지 컬럼 추가
train['0'] = 0
train['1'] = 0
train['2'] = 0
train['3'] = 0


In [4]:
# 문제 clustering

## '문제' 컬럼이 '?'로 끝나는 행들의 'cluster' 값을 1로 변경
train.loc[train['문제'].str.endswith('?'), 'cluster'] = 1
## '문제' 컬럼이 '시나리오 1 |'을 포함하는 행들의 'cluster' 값을 2로 변경
train.loc[train['문제'].str.contains('시나리오 1 |', regex=False), 'cluster'] = 2
## '문제' 컬럼이 '___'을 포함하는 행들의 'cluster' 값을 3으로 변경 -> 빈공간을 채워야 함
train.loc[train['문제'].str.contains('___', regex=False), 'cluster'] = 3
## 명제(진술)이 참인지 거짓인지 판단하는 문제
train.loc[train['문제'].str.contains('명제 1 |',regex=False), 'cluster'] = 4
## 정보를 읽고 그에 맞게 답하는 문제
train.loc[train['문제'].str.contains('다음 정보', regex=False), 'cluster'] = 5

# 3번문제 한정 데이터 전처리 : ___ -> <BLANK>
subset = train.loc[train['cluster'] == 3, '문제']
train.loc[train['cluster'] == 3, '문제'] = subset.str.replace(r'_{2,}', '<BLANK>', regex=True)



In [5]:
train['cluster'].value_counts().sort_index()

,count
cluster,
0,1659
1,4482
2,627
3,1768
4,41
5,423


In [ ]:
# 선택지 리스트화 + 리스트화된 선택지를 항목별로 쪼갬 + 각 번호에 맞는 컬럼에 넣음

for index, row in train.iterrows():
  try:
    train['__선택지'][index] = ast.literal_eval(train['선택지'][index])
  except Exception as e:
    print(f'{index} error 발생 : {e}')
    print("원본 선택지:", train['선택지'][index])
    # 에러 발생 시 원본 선택지를 그대로 사용
    train['__선택지'][index] = train['선택지'][index]

  try:
    train['0'][index] = train['__선택지'][index][0]
  except:
    train['0'][index] = '0'

  try:
    train['1'][index] = train['__선택지'][index][1]
  except:
    train['1'][index] = '0'

  try:
    train['2'][index] = train['__선택지'][index][2]
  except:
    train['2'][index] = '0'

  try:
    train['3'][index] = train['__선택지'][index][3]
  except:
    train['3'][index] = '0'


In [7]:
# 디버깅용 문항 개수 컬럼 추가 (train은 7개 빼고 전부,  test는 전부 이상 없음)
train['문항개수'] = train['__선택지'].apply(lambda x: len(x))

In [8]:
train

,문제,선택지,답안,cluster,__선택지,0,1,2,3,문항개수
0,다음 중 상관 계수 r의 값을 변경할 수 있는 것은 무엇입니까?,"['측정 단위의 변경', '어느 변수를 x로, 어느 변수를 y로 부르는지의 변경',...",3,1,"[측정 단위의 변경, 어느 변수를 x로, 어느 변수를 y로 부르는지의 변경, x 변...",측정 단위의 변경,"어느 변수를 x로, 어느 변수를 y로 부르는지의 변경",x 변수의 모든 값에 동일한 상수를 더하는 것,위의 어느 것도 r 값을 변경할 수 없다.,4
1,한 건물 소유자가 도시에 있는 큰 건물을 소유하고 있었습니다. 소유자는 건물을 상인...,"['건물 소유자와 상인 간에는 부동산 소유권과 계약상의 관계가 존재합니다.', '상...",2,1,"[건물 소유자와 상인 간에는 부동산 소유권과 계약상의 관계가 존재합니다., 상인과 ...",건물 소유자와 상인 간에는 부동산 소유권과 계약상의 관계가 존재합니다.,상인과 법인 간에는 부동산 소유권과 계약상의 관계가 존재합니다.,건물 소유자와 법인 간에는 부동산 소유권 관계는 존재하지만 계약상의 관계는 존재하지...,건물 소유자와 비영리 단체 간에는 부동산 소유권 관계도 계약상의 관계도 존재하지 않...,4
2,유전율이 4.0인 비자성 유전체 내부에서의 빛의 속도는 <BLANK>입니다.,"['1.2 x 10^9 m/s', '3.0 x 10^8 m/s', '1.5 x 10...",2,3,"[1.2 x 10^9 m/s, 3.0 x 10^8 m/s, 1.5 x 10^8 m/...",1.2 x 10^9 m/s,3.0 x 10^8 m/s,1.5 x 10^8 m/s,1.0 x 10^8 m/s,4
3,다음 중 완전 경쟁 하에서 가격(P)이 한계 수익(MR)과 같은 이유와 독점 또는 ...,"['I, II, III.', 'I와 II.', 'I만.', 'I와 III.']",1,0,"[I, II, III., I와 II., I만., I와 III.]","I, II, III.",I와 II.,I만.,I와 III.,4
4,다음 중 어떤 것이 일부 지역에서 일반적으로 큰 조수 범위(최대 60피트)가 있고 ...,"['태양의 위치가 지역마다 다릅니다.', '코리올리 효과와 지구의 자전이 고위도에서...",2,1,"[태양의 위치가 지역마다 다릅니다., 코리올리 효과와 지구의 자전이 고위도에서 조수...",태양의 위치가 지역마다 다릅니다.,코리올리 효과와 지구의 자전이 고위도에서 조수 흐름을 증대시키는 경향이 있습니다.,해저 지형과 해안선의 모양이 특정 지역에서 조수 흐름을 증폭시키는 역할을 합니다.,무역풍이 물을 바위 해안 근처의 큰 조수 융기로 밀어냅니다.,4
...,...,...,...,...,...,...,...,...,...,...
8995,원형 지구 궤도의 위성은 일정한 속도를 유지합니다. 그 이유는 중력이 <BLANK>...,"['제로', '사실상 제로', '지구 표면보다 상당히 약하다', '원형 경로를 따라...",3,3,"[제로, 사실상 제로, 지구 표면보다 상당히 약하다, 원형 경로를 따라 힘의 성분이...",제로,사실상 제로,지구 표면보다 상당히 약하다,원형 경로를 따라 힘의 성분이 없기 때문이다,4
8996,한 사람이 리모컨을 사용하여 텔레비전의 볼륨 수준을 변경합니다. 리모컨에서 텔레비전...,"['가시광선 펄스', '라디오파 신호', '일련의 적외선 펄스', '일련의 강한 적...",2,3,"[가시광선 펄스, 라디오파 신호, 일련의 적외선 펄스, 일련의 강한 적외선 빔]",가시광선 펄스,라디오파 신호,일련의 적외선 펄스,일련의 강한 적외선 빔,4
8997,직각삼각형의 빗변이 10인치이고 한 각이 $45^{\circ}$입니다. 삼각형의 면...,"['10', '45', '125', '25']",3,1,"[10, 45, 125, 25]",10,45,125,25,4
8998,자연주의와 실증주의의 주요 차이점은 무엇입니까?,"['한편으로 자연주의는 보편적이고 객관적인 범위의 규칙들에 기초하고, 다른 한편으로...",0,1,"[한편으로 자연주의는 보편적이고 객관적인 범위의 규칙들에 기초하고, 다른 한편으로 ...","한편으로 자연주의는 보편적이고 객관적인 범위의 규칙들에 기초하고, 다른 한편으로 실...","자연주의는 인권법 이론에만 봉사하는 반면, 실증주의는 더 넓고 일반적인 범위를 가진다","자연주의는 국제 사회의 근본적인 가치를 다루고, 실증주의는 국제 규칙 그 자체를 다룬다","자연주의는 유럽 외부에서 옹호되는 사상 학파이며, 실증주의는 유럽 중심의 이론이다",4


In [9]:
train['문항개수'].value_counts()

,count
문항개수,
4,8993
592,1
112,1
3,1
143,1
139,1
162,1
271,1


In [10]:
train.to_csv('train_preprocessing.csv', index=False, encoding='utf-8')

## 문제 항목별 Train Data 체크

In [11]:
# train[train['cluster'] == 0]['문제'].tolist()
train[train['cluster'] == 0][['문제', '__선택지']]

,문제,__선택지
3,다음 중 완전 경쟁 하에서 가격(P)이 한계 수익(MR)과 같은 이유와 독점 또는 ...,"[I, II, III., I와 II., I만., I와 III.]"
12,"성희롱의 맥락에서 ""quid pro quo""는 다음을 의미합니다:","[직원이 성적인 요구에 대한 반응이 그의/그녀의 직업에 영향을 미치는 상황., ""성..."
23,"경제가 경기 침체 격차를 겪고 있다면, 격차를 해소하기 위한 적절한 재정 정책과 그...","[세금 인상. 증가 감소, 지출 감소. 감소 증가, 세..."
26,마그나 카르타는,"[1689년에 의회에서 언론의 자유를 보장한 협정, 1215년에 왕과 귀족들 사이에..."
34,간접 진리표를 사용하여 다음 논증이 타당한지 여부를 결정하십시오. 논증이 타당하지 ...,"[타당함, 타당하지 않음. B, C, D가 참이고 A가 거짓일 때 반례, 타당하지 ..."
...,...,...
8987,1차 데이터는,"[마케팅 연구를 위해 처음으로 수집된 데이터입니다., 2차 데이터보다 획득하는 데 ..."
8988,정신과 의사가 7년 전에 평가된 환자에 대한 보고서 사본을 요청하는 서명된 환자 정...,[시간이 너무 많이 경과했기 때문에 보고서를 보내기를 거부하는 편지를 정신과 의사에...
8990,이 장에서 강조된 가장 중요한 점 중 하나는 신체 변화가,"[모든 사람에게 발생한다, 일부 사람들에게 더 많이 발생하고 다른 사람들에게는 덜 ..."
8993,"W. E. B. Dubois에 따르면, 노예제 폐지 후 아프리카계 미국인의 진보는",[가장 재능 있는 백인 미국인이 접근할 수 있었던 것과 같은 철저하고 광범위한 교육...


In [12]:
train[train['cluster'] == 0]['문제'].tolist()
# train[train['cluster'] == 0][['문제', '__선택지']]

['다음 중 완전 경쟁 하에서 가격(P)이 한계 수익(MR)과 같은 이유와 독점 또는 불완전 경쟁 하에서 가격(P)이 한계 수익보다 큰 이유를 올바르게 설명한 것은 무엇입니까? I. 완전 경쟁에서는 MR = P = 수요가 수평 함수입니다. II. 불완전 경쟁에서는 P > MR이 하향 경사 함수로 나타납니다. III. 불완전 경쟁에서는 수요와 가격이 수직 함수로 나타납니다.',
 '성희롱의 맥락에서 "quid pro quo"는 다음을 의미합니다:',
 '경제가 경기 침체 격차를 겪고 있다면, 격차를 해소하기 위한 적절한 재정 정책과 그 정책이 실질 GDP와 실업률에 미치는 예상 영향을 아래에서 선택하십시오. 재정 정책 \xa0\xa0\xa0 실질 GDP \xa0\xa0\xa0 실업률',
 '마그나 카르타는',
 '간접 진리표를 사용하여 다음 논증이 타당한지 여부를 결정하십시오. 논증이 타당하지 않다면 반례를 제시하는 옵션을 선택하십시오. (다른 반례가 있을 수도 있습니다.)\n(A ∨ B) ⊃ C\nC ⊃ ~D / D ⊃ A',
 '집합 $$\\{1,2,3,\\ldots,100\\}$$에서 무작위로 선택된 정수가 2로 나누어지고 3으로 나누어지지 않을 확률은 얼마입니까? 답을 일반 분수로 표현하십시오.',
 '한 직원이 회사의 직원 지원 프로그램을 통해 심리학자로부터 상담을 받고 있습니다. 이 상황에서 심리학자는 직원이 치료를 받고 있다는 사실을 상사에게 알릴 수 있습니다:',
 '목성의 벨트와 존은',
 '특정 로스쿨 지원자의 평균 법학적성시험(LSAT) 점수는 650이고 표준 편차는 45입니다. 700점 이상의 점수를 받은 지원자만 고려한다고 가정할 때, 고려된 지원자 중 740점 이하의 점수를 받은 지원자의 비율은 얼마입니까? (점수가 정규 분포한다고 가정하십시오.)',
 '다음 문장을 주어진 PL 공식 중 가장 잘 상징화한 것은 무엇입니까?\n의사들은 환자들이 무례하지 않는 한 도움이 된다.',
 '나이 든 근로자와 젊은 근로자가 직장을 잃었을 때, 나이

In [13]:
# train[train['cluster'] == 1]['문제'].tolist()
train[train['cluster'] == 1][['문제', '__선택지']]

,문제,__선택지
0,다음 중 상관 계수 r의 값을 변경할 수 있는 것은 무엇입니까?,"[측정 단위의 변경, 어느 변수를 x로, 어느 변수를 y로 부르는지의 변경, x 변..."
1,한 건물 소유자가 도시에 있는 큰 건물을 소유하고 있었습니다. 소유자는 건물을 상인...,"[건물 소유자와 상인 간에는 부동산 소유권과 계약상의 관계가 존재합니다., 상인과 ..."
4,다음 중 어떤 것이 일부 지역에서 일반적으로 큰 조수 범위(최대 60피트)가 있고 ...,"[태양의 위치가 지역마다 다릅니다., 코리올리 효과와 지구의 자전이 고위도에서 조수..."
5,1986년 리비아 지도자 무아마르 카다피가 '죽음의 선'이라고 부른 선을 그은 해역...,"[시드라만, 오만만, 홍해, 페르시아만]"
6,다음 중 주로 화학적 감각은 무엇입니까?,"[촉각, 시각, 미각, 청각]"
...,...,...
8991,라이엘과 다윈 이전에 지배적이었던 개념은 무엇이었습니까?,"[지구는 몇 천 년 되었고, 개체군은 변하지 않는다., 지구는 몇 천 년 되었고, ..."
8992,햇빛 아래서 파란색으로 보이는 종이가 녹색 필터를 통과한 빨간색 빛만으로 비춰졌을 ...,"[파란색, 녹색, 빨간색, 검은색]"
8997,직각삼각형의 빗변이 10인치이고 한 각이 $45^{\circ}$입니다. 삼각형의 면...,"[10, 45, 125, 25]"
8998,자연주의와 실증주의의 주요 차이점은 무엇입니까?,"[한편으로 자연주의는 보편적이고 객관적인 범위의 규칙들에 기초하고, 다른 한편으로 ..."


In [14]:
train[train['cluster'] == 1]['문제'].tolist()
# train[train['cluster'] == 1][['문제', '__선택지']]

['다음 중 상관 계수 r의 값을 변경할 수 있는 것은 무엇입니까?',
 '한 건물 소유자가 도시에 있는 큰 건물을 소유하고 있었습니다. 소유자는 건물을 상인에게 월 $10,000의 임대료로 20년 동안 임대했습니다. 건물 소유자와 상인 간의 임대차 계약서에는 상인이 "잘 알려진 신용 평가 회사에서 \'A\' 신용 등급을 받은 법인 외에는 이 임대차를 양도할 수 없다"고 명시되어 있었습니다. 한 달 후, 상인은 필요한 신용 등급을 갖추지 않은 법인에게 건물을 임대했습니다. 상인-법인 임대는 5년 동안 월 $15,000의 임대료로 법인이 상인에게 지불하는 조건이었습니다. 또한, 법인은 "건물 소유자와 상인 간의 임대차 계약의 모든 조건을 준수할 것"에 동의했습니다. 1년 후, 법인은 상인-법인 임대의 남은 기간 동안 비영리 단체에게 건물을 임대했습니다. 비영리 단체는 법인이 건물을 비우는 당일에 해당 건물을 점유했습니다. 법인-비영리 임대차 계약에 따라, 비영리 단체는 법인에게 월 $17,500의 임대료를 직접 지불해야 했습니다. 비영리 단체는 잘 알려진 신용 평가 회사에서 \'B\' 신용 등급을 가지고 있습니다. 다음 중 다양한 당사자 간의 법적 관계에 대해 올바르지 않은 것은 무엇입니까?',
 '다음 중 어떤 것이 일부 지역에서 일반적으로 큰 조수 범위(최대 60피트)가 있고 다른 지역에서는 1~2피트의 조수 범위를 가지는 이유를 가장 잘 설명합니까?',
 "1986년 리비아 지도자 무아마르 카다피가 '죽음의 선'이라고 부른 선을 그은 해역은 어디입니까?",
 '다음 중 주로 화학적 감각은 무엇입니까?',
 '다음 중 자연 선택의 추론에 해당하는 진술은 무엇입니까?',
 '다음 중 어느 장애가 빛에 민감한 사람이 노출되는 빛의 양과 가장 관련이 있을 가능성이 높습니까?',
 '마누엘은 중고차를 6,049달러에 샀습니다. 마누엘이 산 중고차의 가격을 숫자로 쓰면 얼마입니까?',
 '다음 중 _tu quoque_ 오류를 설명하는 것은 무엇입니까?',
 '케이코는

In [15]:
# train[train['cluster'] == 2]['문제'].tolist()
train[train['cluster'] == 2][['문제', '__선택지']]

,문제,__선택지
8,다음 두 가지 시나리오 중 어느 것에서 주인공(나/저/내가 사용하는)이 2020년 ...,"[잘못, 잘못, 잘못, 잘못 아님, 잘못 아님, 잘못, 잘못 아님, 잘못 아님]"
13,"2020년 기준 미국의 일반적인 도덕 기준에 따르면, 이 두 시나리오 중 어느 시나...","[잘못됨, 잘못됨, 잘못됨, 잘못되지 않음, 잘못되지 않음, 잘못됨, 잘못되지 않음..."
16,"2020년 현재 미국의 일반적인 도덕 기준에 따르면, 이 두 가지 시나리오 중 어느...","[잘못됨, 잘못됨, 잘못됨, 잘못되지 않음, 잘못되지 않음, 잘못됨, 잘못되지 않음..."
28,다음 두 시나리오 중 어느 시나리오에서 주인공(나/내가/내가 사용)의 행동이 202...,"[잘못, 잘못, 잘못, 잘못 아님, 잘못 아님, 잘못, 잘못 아님, 잘못 아님]"
38,"2020년 현재 미국의 일반적인 도덕 기준에 따르면, 다음 두 시나리오 중 어느 시...","[잘못됨, 잘못됨, 잘못됨, 잘못되지 않음, 잘못되지 않음, 잘못됨, 잘못되지 않음..."
...,...,...
8908,이 두 시나리오 중 어느 시나리오에서 주인공(나/내가/나의)이 2020년 미국의 일...,"[잘못됨, 잘못됨, 잘못됨, 잘못되지 않음, 잘못되지 않음, 잘못됨, 잘못되지 않음..."
8919,"2020년 미국의 일반적인 도덕 기준에 따르면, 이 두 시나리오 중 어느 시나리오에...","[잘못됨, 잘못됨, 잘못됨, 잘못되지 않음, 잘못되지 않음, 잘못됨, 잘못되지 않음..."
8938,이 두 가지 시나리오 중 어느 것이 2020년 미국의 일반적인 도덕 기준에 따라 주...,"[잘못, 잘못, 잘못, 잘못 아님, 잘못 아님, 잘못, 잘못 아님, 잘못 아님]"
8976,이 두 가지 시나리오 중 어느 것에서 주인공(나/내가/내) 이 2020년 미국의 일...,"[잘못, 잘못, 잘못, 잘못 아님, 잘못 아님, 잘못, 잘못 아님, 잘못 아님]"


In [16]:
train[train['cluster'] == 2]['문제'].tolist()
# train[train['cluster'] == 2][['문제', '__선택지']]

['다음 두 가지 시나리오 중 어느 것에서 주인공(나/저/내가 사용하는)이 2020년 미국의 일반적인 도덕적 기준에 따르면 명백히 도덕적으로 잘못된 행동을 했습니까? 시나리오 1 | 동물원을 보는 것이 슬퍼서 우리 도시 동물원의 동물들을 풀어주기로 결정했습니다. 시나리오 2 | 영화가 상영되는 동안 공공 극장에서 친구에게 잠시 속삭였습니다.',
 '2020년 기준 미국의 일반적인 도덕 기준에 따르면, 이 두 시나리오 중 어느 시나리오에서 주인공(나/내가/내)은 명백히 도덕적으로 잘못된 행동을 하나요? 시나리오 1 | 나는 자동차의 브레이크를 고장 냈다. 시나리오 2 | 나는 생일 선물로 할머니가 주신 수표를 현금화하기 전에 내 이름을 뒷면에 추가했다.',
 '2020년 현재 미국의 일반적인 도덕 기준에 따르면, 이 두 가지 시나리오 중 어느 시나리오에서 주인공(나/내가/내가 사용하는)이 명백히 도덕적으로 잘못된 행동을 합니까? 시나리오 1 | 퇴근 후, 나는 저녁으로 먹기 위해 내 아기를 요리하기로 결정했다. 시나리오 2 | 나는 장난꾸러기 아이를 구석에 세워 놓고 반성 시간을 주었다.',
 '다음 두 시나리오 중 어느 시나리오에서 주인공(나/내가/내가 사용)의 행동이 2020년 현재 미국의 일반적인 도덕 기준에 따라 명백히 도덕적으로 잘못된 것입니까? 시나리오 1 | 나는 이웃집에서 토론을 시작했다. 시나리오 2 | 나는 내 입술 주사를 위해 남편의 소중한 기타를 전당포에 맡겼다.',
 '2020년 현재 미국의 일반적인 도덕 기준에 따르면, 다음 두 시나리오 중 어느 시나리오에서 주인공(나/내가/나의)이 명백히 도덕적으로 잘못된 행동을 했습니까? 시나리오 1 | 나는 내 가장 친한 친구의 어두운 비밀을 폭로했다. 시나리오 2 | 은행에 갔을 때 당국에 잡히지 않기 위해 얼굴을 마스크로 가렸다.',
 '다음 두 가지 시나리오 중 어느 시나리오에서 주인공(나/내가/나의 사용)이 2020년 미국의 일반적인 도덕 기준에 따라 명백히 도덕적으로 잘못된 행동을 하는가?

In [17]:
# train[train['cluster'] == 3]['문제'].tolist()
train[train['cluster'] == 3][['문제', '__선택지']]

,문제,__선택지
2,유전율이 4.0인 비자성 유전체 내부에서의 빛의 속도는 <BLANK>입니다.,"[1.2 x 10^9 m/s, 3.0 x 10^8 m/s, 1.5 x 10^8 m/..."
9,불감성 손실은 <BLANK>에서 발생하지 않습니다.,"[발열., 비가습 산소 사용., 과도한 체중 감소., 설사.]"
10,뉴런 막 전위가 +50 밀리볼트에서 -70 밀리볼트로 변화하는 것은 <BLANK>로...,"[탈분극, 재분극, 과분극, 활동 전위]"
24,새벽 3시입니다. 당신은 남성입니다. 소변을 보기 위해 잠에서 깼습니다. 변기에 서...,"[성기 포진, 에이즈, 매독, 임질]"
30,"공기 주입 파티 풍선을 압착할 때, 그 <BLANK>이 증가합니다.","[부피, 질량, 무게, 밀도]"
...,...,...
8952,"<BLANK> 법학파의 철학자들은 법이 사회의 도덕성, 역사, 논리 또는 사회학의 ...","[명령, 법과 경제학, 자연법, 비판적 법학]"
8957,GDP는 각 <BLANK>에 대해 계산됩니다,"[분기; 경제 분석국, 주; 경제 분석국, 월; 경제 분석국, 월; 노동 통계국]"
8959,운동 후 회복 중 인산 크레아틴의 재합성은 <BLANK>에 의해 억제됩니다.,"[크레아틴 과다., 과호흡., 산소 과다., 산소 부족.]"
8995,원형 지구 궤도의 위성은 일정한 속도를 유지합니다. 그 이유는 중력이 <BLANK>...,"[제로, 사실상 제로, 지구 표면보다 상당히 약하다, 원형 경로를 따라 힘의 성분이..."


In [18]:
train[train['cluster'] == 3]['문제'].tolist()
# train[train['cluster'] == 3][['문제', '__선택지']]

['유전율이 4.0인 비자성 유전체 내부에서의 빛의 속도는 <BLANK>입니다.',
 '불감성 손실은 <BLANK>에서 발생하지 않습니다.',
 '뉴런 막 전위가 +50 밀리볼트에서 -70 밀리볼트로 변화하는 것은 <BLANK>로 간주됩니다.',
 '새벽 3시입니다. 당신은 남성입니다. 소변을 보기 위해 잠에서 깼습니다. 변기에 서 있는 동안 극심한 고통을 느낍니다. 당신의 음경이 불타는 것처럼 느껴집니다. 소변을 본 후 음경에서 노란색 분비물이 나오는 것을 발견합니다. 당신의 신음 소리에 배우자가 깨어났고, 그녀는 화장실로 달려와 무슨 일이냐고 묻습니다. 당신은 그녀에게 아마도 <BLANK>에 걸린 것 같다고 말합니다.',
 '공기 주입 파티 풍선을 압착할 때, 그 <BLANK>이 증가합니다.',
 '개의 입에 소량의 고기 가루를 넣은 것은 파블로프의 <BLANK>였습니다.',
 '페닐케톤뇨증(PKU)과 관련된 정신 지체는 <BLANK>로 예방할 수 있습니다.',
 '일출이나 일몰의 붉은색은 주로 <BLANK> 않은 빛에 기인합니다.',
 '마추픽추는 산에 <BLANK>로 지어졌습니다.',
 '화학 방어 기제를 가진 동물들이 채택하는 경고 색채는 <BLANK>입니다.',
 'xy-평면의 제1사분면에서 곡선 y = x와 y = x^2로 둘러싸인 영역을 y축을 중심으로 회전시킬 때, 생성되는 회전체의 부피는 <BLANK> 입니다.',
 '전력계의 이동 코일 회로의 저항은 <BLANK>이어야 합니다.',
 '미국 정부의 사법부는 법을 <BLANK> 권한을 가지고 있습니다.',
 '고고학적 및 유전적 증거에 따르면 늑대와 개의 분리가 포함된 가축화 과정은 약 <BLANK> 전에 발생했습니다.',
 'Scott은 NEBOTYA라는 글자를 20분 동안 단어로 만들기 위해 시도했지만 성공하지 못했습니다. 수업에 가는 도중, 갑자기 그 단어가 BAYONET이라는 답이 떠올랐습니다. 이것은 <BLANK>의 예입니다.',
 '1세 남아가 손과 발에 물집이 생긴 신선한 화상을 

In [19]:
train[train['cluster'] == 4][['문제', '__선택지']]

,문제,__선택지
104,명제 1 | 모든 양의 정수 계수에 대해 자유 아벨 군이 존재한다. 명제 2 | 유...,"[참, 참, 거짓, 거짓, 참, 거짓, 거짓, 참]"
533,"명제 1 | H가 G의 부분군이고 a가 G에 속하면, aH는 a가 H에 속할 때 그...","[참, 참, 거짓, 거짓, 참, 거짓, 거짓, 참]"
619,명제 1 | 비가환군의 몫군은 비가환군이다. 명제 2 | K가 H의 정규 부분군이고...,"[참, 참, 거짓, 거짓, 참, 거짓, 거짓, 참]"
836,명제 1 | 소수 p에 대해 차수가 p^2인 모든 군은 아벨 군이다. 명제 2 | ...,"[참, 참, 거짓, 거짓, 참, 거짓, 거짓, 참]"
838,명제 1 | 4x - 2는 Z에서 기약적이다. 명제 2 | 4x - 2는 Q에서 기...,"[참, 참, 거짓, 거짓, 참, 거짓, 거짓, 참]"
928,"명제 1 | R이 정역이라면, R[x]도 정역이다. 명제 2 | R이 환이고 f(x...","[참, 참, 거짓, 거짓, 참, 거짓, 거짓, 참]"
1625,명제 1 | 만약 T: V -> W가 선형 변환이고 dim(V) < dim(W) <...,"[참, 참, 거짓, 거짓, 참, 거짓, 거짓, 참]"
1633,명제 1 | 순환군의 동형상은 순환군이다. 명제 2 | 아벨 군의 동형상은 아벨 군이다.,"[참, 참, 거짓, 거짓, 참, 거짓, 거짓, 참]"
1978,명제 1 | 부분환의 단위는 환의 단위와 동일해야 합니다. 명제 2 | 모든 체는 ...,"[참, 참, 거짓, 거짓, 참, 거짓, 거짓, 참]"
2057,명제 1 | 모든 자유 아벨 군은 무비틀림이다. 명제 2 | 모든 유한 생성 무비틀...,"[참, 참, 거짓, 거짓, 참, 거짓, 거짓, 참]"


In [20]:
train[train['cluster'] == 4]['문제'].tolist()
# train[train['cluster'] == 4][['문제', '__선택지']]

['명제 1 | 모든 양의 정수 계수에 대해 자유 아벨 군이 존재한다. 명제 2 | 유한 생성 아벨 군은 베티 수가 생성 집합의 원소 수와 같을 경우 자유 아벨 군이다.',
 '명제 1 | H가 G의 부분군이고 a가 G에 속하면, aH는 a가 H에 속할 때 그리고 그 경우에만 G의 부분군이다. 명제 2 | H가 G의 부분군이고 a와 b가 G에 속하면, aH = bH는 ab가 H에 속할 때 그리고 그 경우에만 성립한다.',
 '명제 1 | 비가환군의 몫군은 비가환군이다. 명제 2 | K가 H의 정규 부분군이고 H가 G의 정규 부분군이면, K는 G의 정규 부분군이다.',
 '명제 1 | 소수 p에 대해 차수가 p^2인 모든 군은 아벨 군이다. 명제 2 | 고정된 소수 p에 대해, 군 G의 Sylow p-부분군이 G의 유일한 Sylow p-부분군일 때, 그리고 그럴 때에만 G의 정규 부분군이다.',
 '명제 1 | 4x - 2는 Z에서 기약적이다. 명제 2 | 4x - 2는 Q에서 기약적이다.',
 '명제 1 | R이 정역이라면, R[x]도 정역이다. 명제 2 | R이 환이고 f(x)와 g(x)가 R[x]에 있다면, deg (f(x)g(x)) = deg f(x) + deg g(x)이다.',
 '명제 1 | 만약 T: V -> W가 선형 변환이고 dim(V) < dim(W) < 1이라면, T는 단사여야 한다. 명제 2 | dim(V) = n이고 T: V -> V가 선형이라고 가정하자. 만약 T가 단사라면, 그것은 전사이다.',
 '명제 1 | 순환군의 동형상은 순환군이다. 명제 2 | 아벨 군의 동형상은 아벨 군이다.',
 '명제 1 | 부분환의 단위는 환의 단위와 동일해야 합니다. 명제 2 | 모든 체는 정역입니다.',
 '명제 1 | 모든 자유 아벨 군은 무비틀림이다. 명제 2 | 모든 유한 생성 무비틀림 아벨 군은 자유 아벨 군이다.',
 '명제 1 | 환 준동형이 일대일이 되기 위한 필요충분조건은 핵이 {0}인 경우이다. 명제 2 | Q는 R의 아이디얼이다.'

In [21]:
train.loc[train['문제'].str.contains('다음 정보', regex=False)][['문제', '__선택지']] # 423

,문제,__선택지
111,이 질문은 다음 정보와 관련이 있습니다.\n질문에 답하기 위해 다음 인용문을 읽으십...,"[독일 역사 맥락에서의 산업 혁명, 1848년 혁명의 실패, 독일에서 나치당의 부상..."
113,다음 정보를 참조하십시오.\n우리는 이 새로운 영토를 어떻게 통치할 것인지 묻습니다...,"[대중주의, 영국과의 전쟁, 미국의 헌법 개혁, 미국의 해외 제국 창설]"
114,다음 정보에 관한 질문입니다.\n[T]he the end and measure of...,"[신성왕권, 절대주의, 입헌주의, 사회주의]"
130,"다음 정보와 관련된 질문입니다.\n""중국 황제 폐하께서는 영국 신민들이 가족과 함께...","[식민지, 영향권, 보호령, 사업체]"
155,"다음 정보를 참조하여 질문에 답하십시오.\n""포르투갈 사람들이 중국의 마카오에서 일...","[중앙아시아, 유럽, 중동, 신세계]"
...,...,...
8926,다음 정보에 관한 질문입니다.\n아래의 발췌문을 읽으십시오.\n이 부패는 바울에 의...,"[칼빈주의자들은 국가가 종교 조직에 절대적으로 종속된다고 믿었습니다., 칼빈주의자들..."
8947,"다음 정보와 관련된 질문입니다.\n""다음 3월 4일에 이 당 [공화당]이 정부를 장...",[남부 민주당은 사우스캐롤라이나에 대한 군사 행동을 중지시키기 위해 의회의 권한에 ...
8949,"다음 정보와 관련된 질문입니다.\n""당신의 감정은 우리의 사태가 급속히 위기로 치닫...","[현대의 이신론적 영적 신념은 ""원죄""에 대한 칼뱅주의적 개념을 포기함으로써 잘못된..."
8973,"다음 정보와 관련된 질문입니다.\n""오늘날 이 나라 사람들 중 절반은 불공정한 법을...","[셔먼 반독점법 (1890)., 주간 통상법 (1887)., 연방 무역위원회법 (1..."


In [22]:
train.loc[train['문제'].str.contains('다음 정보', regex=False)]['문제'].tolist()

['이 질문은 다음 정보와 관련이 있습니다.\n질문에 답하기 위해 다음 인용문을 읽으십시오.\n1848년은 독일 역사, 나아가 유럽 역사에서 결정적인 해였습니다: 이 해는 독일의 과거를 요약하고 독일의 미래에 영감을 주었습니다. … 무한한 아이디어의 힘에 대한 믿음으로 이렇게 영감을 받은 혁명은 없었습니다; 결과적으로 아이디어의 힘을 이렇게 불신하게 만든 혁명도 없었습니다. 혁명의 성공은 보수적인 아이디어를 불신하게 만들었고; 혁명의 실패는 자유주의적인 아이디어를 불신하게 만들었습니다. 그 이후로, 남은 것은 힘의 아이디어뿐이었으며, 이 아이디어는 그때부터 독일 역사의 키를 잡았습니다. 1521년 이후 처음으로 독일 국민은 독일 무대의 중심에 섰지만 다시 한 번 그들의 신호를 놓쳤습니다. 독일 역사는 전환점을 맞이했지만 전환하지 못했습니다. 이것이 1848년의 숙명적인 본질이었습니다.\nA. J. P. 테일러, 독일 역사 강의, 1945\n테일러의 분석에서 이 인용문의 주제는',
 '다음 정보를 참조하십시오.\n우리는 이 새로운 영토를 어떻게 통치할 것인지 묻습니다. 저는 대답합니다: 지역 조건과 상황의 필요성에 따라 통치 방법이 발전할 것입니다. 영국이 외국을 통치할 수 있다면, 미국도 할 수 있습니다. 독일이 외국을 통치할 수 있다면, 미국도 할 수 있습니다. 그들이 보호국을 감독할 수 있다면, 미국도 할 수 있습니다. 하와이를 통치하는 것이 뉴멕시코나 캘리포니아를 통치하는 것보다 왜 더 어렵습니까? 둘 다 야만적이고 이질적인 인구를 가지고 있었고, 둘 다 우리 지배 하에 들어왔을 때 오늘날 필리핀보다 정부 중심지에서 더 멀리 떨어져 있었습니다.\n당신의 투표로 미국의 통치 능력이 쇠퇴했다고 말할 것입니까? 자치 경험의 한 세기가 결과를 내지 못했다고 말할 것입니까? 당신의 투표로 미국의 힘과 실용적 감각에 불신을 표명할 것입니까? 아니면 우리가 통치의 피를 가지고 있고, 지배의 심장을 가지고 있으며, 행정의 두뇌와 천재성을 가지고 있다고 말할 것입니까? 

In [23]:
print(type(train['선택지'][0]))
print(type(train['__선택지'][0]))

<class 'str'>
<class 'list'>


In [24]:
cluster_prompts = {
        0: ("Cluster 0 : Please carefully read the following question. Based on your understanding, knowledge, and reasoning, choose the most appropriate answer or provide the best possible explanation.\n"
        "Cluster 0 : 다음 질문을 신중하게 읽고, 자신의 이해, 지식, 그리고 논리를 바탕으로 가장 적절한 답을 선택하거나 최선의 설명을 제시하세요.\n"),
        1: ("Cluster 1 : Please read the following question carefully and choose the most accurate answer based on your understanding and knowledge.\n"
        "Cluster 1 : 다음 질문을 주의 깊게 읽고, 자신의 이해와 지식을 바탕으로 가장 정확한 답을 선택하세요.\n"),
        2: ("Cluster 2 : Consider the moral implications of the following scenarios and choose the one where the action is most clearly wrong according to common moral standards.\n"
        "Cluster 2 : 다음 시나리오들의 도덕적 함의를 고려하고, 일반적인 도덕 기준에 따라 가장 명백히 잘못된 행동이 있는 시나리오를 선택하세요.\n"),
        3: ("Cluster 3 : Complete the following statement by choosing the most appropriate term or concept from the options provided.\n"
        "Cluster 3 : 다음 문장을 완성하기 위해 주어진 선택지 중 가장 적절한 용어 또는 개념을 선택하세요.\n"),
        4: ("Cluster 4 : Evaluate the truth of the following propositions and determine which statements are true and which are false.\n"
        "Cluster 4 :다음 명제들의 진위를 평가하고, 참과 거짓을 판단하세요."),
        5: ("Cluster 5 : Based on the information provided in the following sources, select the most accurate answer to the question posed.\n"
        "Cluster 5 :  제공된 자료의 정보를 바탕으로, 제시된 질문에 가장 정확한 답을 선택하세요."),
}

In [25]:
for idx, row in train.iterrows():
  cluster = row['cluster']
  num = 0

  print(f'{idx} 문제 | 문제 분류 : {cluster}')
  prompt = cluster_prompts.get(cluster, "Provide me an answer to the following question.")
  print(prompt)
  num += 1

  if num > 9:
    print('test complete')
    break

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Cluster 0 : 다음 질문을 신중하게 읽고, 자신의 이해, 지식, 그리고 논리를 바탕으로 가장 적절한 답을 선택하거나 최선의 설명을 제시하세요.

7738 문제 | 문제 분류 : 5
Cluster 5 : Based on the information provided in the following sources, select the most accurate answer to the question posed.
Cluster 5 :  제공된 자료의 정보를 바탕으로, 제시된 질문에 가장 정확한 답을 선택하세요.
7739 문제 | 문제 분류 : 0
Cluster 0 : Please carefully read the following question. Based on your understanding, knowledge, and reasoning, choose the most appropriate answer or provide the best possible explanation.
Cluster 0 : 다음 질문을 신중하게 읽고, 자신의 이해, 지식, 그리고 논리를 바탕으로 가장 적절한 답을 선택하거나 최선의 설명을 제시하세요.

7740 문제 | 문제 분류 : 3
Cluster 3 : Complete the following statement by choosing the most appropriate term or concept from the options provided.
Cluster 3 : 다음 문장을 완성하기 위해 주어진 선택지 중 가장 적절한 용어 또는 개념을 선택하세요.

7741 문제 | 문제 분류 : 1
Cluster 1 : Please read the following question carefully and choose the most accurate answer based on your understanding and knowledge.
Cluster 1 : 다음 질문을 주의 

In [26]:
cluster_prompts

{0: 'Cluster 0 : Please carefully read the following question. Based on your understanding, knowledge, and reasoning, choose the most appropriate answer or provide the best possible explanation.\nCluster 0 : 다음 질문을 신중하게 읽고, 자신의 이해, 지식, 그리고 논리를 바탕으로 가장 적절한 답을 선택하거나 최선의 설명을 제시하세요.\n',
 1: 'Cluster 1 : Please read the following question carefully and choose the most accurate answer based on your understanding and knowledge.\nCluster 1 : 다음 질문을 주의 깊게 읽고, 자신의 이해와 지식을 바탕으로 가장 정확한 답을 선택하세요.\n',
 2: 'Cluster 2 : Consider the moral implications of the following scenarios and choose the one where the action is most clearly wrong according to common moral standards.\nCluster 2 : 다음 시나리오들의 도덕적 함의를 고려하고, 일반적인 도덕 기준에 따라 가장 명백히 잘못된 행동이 있는 시나리오를 선택하세요.\n',
 3: 'Cluster 3 : Complete the following statement by choosing the most appropriate term or concept from the options provided.\nCluster 3 : 다음 문장을 완성하기 위해 주어진 선택지 중 가장 적절한 용어 또는 개념을 선택하세요.\n',
 4: 'Cluster 4 : Evaluate the truth of the following prop

# Train Data tokenizing (미완)

In [27]:
# class CustomDataset:
#   def __init__(self, df, tokenizer, max_seq_length=2048):
#     self.df = df
#     self.tokenizer = tokenizer
#     self.max_seq_length = max_seq_length

#   def tokenizer(self):
#     tokenized_data = []

#     for idx, row in self.df.itterrows():
#       if row['문항개수'] != 4:
#         pair = f'{row['문제']} '

In [28]:
# temp_data = [ '구개 돌기는 <BLANK> 융합되기 시작합니다.',
#               '잉카 문명에서 인신공양은 <BLANK> 전에 행해졌습니다.',
#               '사이리스터의 유지 전류가 2 mA인 경우 래칭 전류는 <BLANK>이어야 합니다.',
#               '치료사가 Rorschach 잉크반점 검사를 사용하여 환자의 문제를 분석했습니다. 그는 아마도 <BLANK>일 것입니다.',
#               '벤은 학생들이 파란색 종이에 인쇄된 질문보다 노란색 종이에 인쇄된 질문에 더 빨리 답할 것이라고 생각합니다. 모든 연구 참가자는 각각 35개의 객관식 질문이 있는 세 가지 테스트를 치를 것입니다. 벤의 실험에서 독립 변수는 <BLANK>입니다.',]

# # tokenizer = AutoTokenizer.from_pretrained("maywell/Synatra-V0.1-7B")

# # if tokenizer.pad_token is None:
# #     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# #     # 패드 토큰 ID 업데이트
# #     tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids('[PAD]')


# # input = tokenizer(temp_data, return_tensors='pt',padding='longest', truncation=True)

# # input

In [29]:
# tokens = tokenizer.decode(2)
# print(tokens)

In [30]:
# # if tokenizer.pad_token is None:
# #     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# #     tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids('[PAD]')

# model = AutoModelForCausalLM.from_pretrained("maywell/Synatra-V0.1-7B")
# tokenizer = tokenizer = AutoTokenizer.from_pretrained("maywell/Synatra-V0.1-7B", trust_remote_code=True)
# tokenizer.eos_token = tokenizer.eos_token
# tokenizer.bos_token = tokenizer.bos_token
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# model.resize_token_embeddings(len(tokenizer))

# # 텍스트를 텐서로 변환
# input = tokenizer(temp_data, return_tensors='pt', padding=True, truncation=True, max_length=2048)

# # 결과 확인
# print(input)

In [31]:
# class CustomDataset:
#     def __init__(self, df, tokenizer):
#         self.df = df
#         self.tokenizer = tokenizer

#     def tokenize(self):
#         tokenized_data = []

#         for idx, row in self.df.iterrows():
#             if row['문항개수'] != 4:
#                 # 문항개수가 4가 아닌 경우: '문제'와 '__선택지'를 엮어서 토크나이징
#                 pair = f"{row['문제']} {row['__선택지']}"
#                 tokenized_pair = self.tokenizer(pair, truncation=True, padding='max_length', max_length=512)
#                 tokenized_data.append(tokenized_pair)
#             else:
#                 # 문항개수가 4인 경우: '문제'와 '0', '1', '2', '3'을 각각 엮어서 토크나이징
#                 for i in range(4):
#                     pair = f"{row['문제']} {row[str(i)]}"
#                     tokenized_pair = self.tokenizer(pair, truncation=True, padding='max_length', max_length=512)
#                     tokenized_data.append(tokenized_pair)

#         return tokenized_data


In [32]:
# # tokenized_data는 이미 토크나이징된 결과물입니다.
# tokenized_data = custom_dataset.tokenize()

# # 모델 입력으로 사용
# input_ids = torch.tensor([data['input_ids'] for data in tokenized_data])
# attention_masks = torch.tensor([data['attention_mask'] for data in tokenized_data])

# # 모델에 입력
# outputs = model(input_ids=input_ids, attention_mask=attention_masks)


In [33]:
train.to_csv('train_preprocessing.csv', index=False, encoding='utf-8')

# Test Data

In [34]:
test = pd.read_csv('test_data.csv')

test['cluster'] = 0
test['__선택지'] = '0'

test['0'] = 0
test['1'] = 0
test['2'] = 0
test['3'] = 0


In [35]:
# 문제 clustering
## '문제' 컬럼이 '?'로 끝나는 행들의 'cluster' 값을 1로 변경
test.loc[test['문제'].str.endswith('?'), 'cluster'] = 1
## '문제' 컬럼이 '시나리오 1 |'을 포함하는 행들의 'cluster' 값을 2로 변경
test.loc[test['문제'].str.contains('시나리오 1 |', regex=False), 'cluster'] = 2
## '문제' 컬럼이 '___'을 포함하는 행들의 'cluster' 값을 3으로 변경 -> 빈공간을 채워야 함
test.loc[test['문제'].str.contains('___', regex=False), 'cluster'] = 3
## 명제(진술)이 참인지 거짓인지 판단하는 문제
test.loc[test['문제'].str.contains('명제 1 |',regex=False), 'cluster'] = 4
## 정보를 읽고 그에 맞게 답하는 문제
test.loc[test['문제'].str.contains('다음 정보', regex=False), 'cluster'] = 5

# 3번문제 한정 데이터 전처리 : ___ -> <BLANK>
subset = test.loc[test['cluster'] == 3, '문제']
test.loc[test['cluster'] == 3, '문제'] = subset.str.replace(r'_{2,}', '<BLANK>', regex=True)



In [ ]:

for index, row in test.iterrows():
  try:
   test['__선택지'][index] = ast.literal_eval(test['선택지'][index])
  except Exception as e:
    print(f'{index} error 발생 : {e}')
    print("원본 선택지:", test['선택지'][index])
    # 에러 발생 시 원본 선택지를 그대로 사용
    test['__선택지'][index] = test['선택지'][index]

  try:
   test['0'][index] = test['__선택지'][index][0]
  except:
   test['0'][index] = '0'

  try:
   test['1'][index] = test['__선택지'][index][1]
  except:
   test['1'][index] = '0'

  try:
   test['2'][index] = test['__선택지'][index][2]
  except:
   test['2'][index] = '0'

  try:
   test['3'][index] = test['__선택지'][index][3]
  except:
   test['3'][index] = '0'

In [37]:
test['문항개수'] = test['__선택지'].apply(lambda x: len(x))

In [38]:
test['문항개수'].value_counts()

,count
문항개수,
4,1000


In [39]:
test['cluster'].value_counts().sort_index()

,count
cluster,
0,188
1,501
2,87
3,212
4,8
5,4


In [40]:
test

,id,문제,선택지,cluster,__선택지,0,1,2,3,문항개수
0,0,샌델은 유전자 강화의 사용이 본질적으로 잘못되었다고 믿습니다. 그 이유는,"['그 사용이 안전하지 않을 것이다.', '그 사용이 불공평할 것이다.', '위의 ...",0,"[그 사용이 안전하지 않을 것이다., 그 사용이 불공평할 것이다., 위의 두 가지 ...",그 사용이 안전하지 않을 것이다.,그 사용이 불공평할 것이다.,위의 두 가지 모두,위의 어느 것도 아니다,4
1,1,유한군 G가 7차 부분군을 포함하지만 (항등원 이외에) 자기 자신의 역원이 되는 원...,"['27', '28', '35', '37']",3,"[27, 28, 35, 37]",27,28,35,37,4
2,2,다음 두 가지 시나리오 중 어느 것이 2020년 미국의 일반적인 도덕 기준에 따라 ...,"['잘못됨, 잘못됨', '잘못됨, 잘못되지 않음', '잘못되지 않음, 잘못됨', '...",2,"[잘못됨, 잘못됨, 잘못됨, 잘못되지 않음, 잘못되지 않음, 잘못됨, 잘못되지 않음...","잘못됨, 잘못됨","잘못됨, 잘못되지 않음","잘못되지 않음, 잘못됨","잘못되지 않음, 잘못되지 않음",4
3,3,"현재 완전 경쟁 가격이 최소 ATC보다 높다면, 장기적으로 다음 중 어떤 일이 발생...","['기업이 산업에 진입하면서 가격이 상승한다.', '기업이 산업에서 퇴출하면서 시장...",1,"[기업이 산업에 진입하면서 가격이 상승한다., 기업이 산업에서 퇴출하면서 시장 균형...",기업이 산업에 진입하면서 가격이 상승한다.,기업이 산업에서 퇴출하면서 시장 균형 수량이 증가한다.,아무 일도 일어나지 않는다. 현재 산업은 장기 균형 상태에 있다.,기업이 산업에 진입하면서 가격이 하락한다.,4
4,4,지구의 계절은 다음 중 무엇에 의해 발생합니까?,"[""지구가 태양 주위를 공전할 때 황도면에 대한 지구 자전축의 기울기"", ""태양흑점...",1,"[지구가 태양 주위를 공전할 때 황도면에 대한 지구 자전축의 기울기, 태양흑점 활동...",지구가 태양 주위를 공전할 때 황도면에 대한 지구 자전축의 기울기,태양흑점 활동의 변동,지구의 공전 궤도가 원이 아닌 타원인 것,24시간 동안의 지구 자전,4
...,...,...,...,...,...,...,...,...,...,...
995,995,Stark 회사는 12월 31일 재무제표 발행 전에 10월에 발생한 사건으로 인한 ...,"['$100,000, $100,000 - $150,000', '$100,000 $1...",3,"[$100,000, $100,000 - $150,000, $100,000 $150,...","$100,000, $100,000 - $150,000","$100,000 $150,000","$0 $100,000 - $150,000","$0 $150,000",4
996,996,20세기 초 서유럽에서 가장 인기 있었던 심리학적 관점은 무엇입니까?,"['신경과학', '진화론적', '행동주의', '정신분석학']",1,"[신경과학, 진화론적, 행동주의, 정신분석학]",신경과학,진화론적,행동주의,정신분석학,4
997,997,그리스 신 제우스의 로마 대응 신은 누구입니까?,"['주피터', '유노', '디오니소스', '넵투누스']",1,"[주피터, 유노, 디오니소스, 넵투누스]",주피터,유노,디오니소스,넵투누스,4
998,998,연구에 따르면 대부분의 사람들은 1쿼트의 물에 1그램의 소금을 맛볼 수 있습니다. ...,"['차이 역치', '절대 역치', '맛의 일관성', '감각 적응']",1,"[차이 역치, 절대 역치, 맛의 일관성, 감각 적응]",차이 역치,절대 역치,맛의 일관성,감각 적응,4


In [41]:
test.to_csv('test_preprocessing.csv', index=False, encoding='utf-8')